# Scraper (Post-Request)

Der Reihe nach:

  * Url (mit post anstatt get) aufrufen.
  * Dabei Post-Info senden
  * Antwort als Json weiterverarbeiten.

For-Loop:

  * Mit einer For-Loop auf das Json-Array zugreifen
  * In der Loop ein Dictionary mit den Angaben die mir interessieren aufsetzen
  * & das Dictionary zu einer Liste hinzufügen.
      * in der For-Loop die sub_url des Eintrags aufrufen (url + id)
      * Mit BeautifulSoup auf die sub_page angucken & die Elemente "legend" abgreifen
      * Das legend-Element das "Herr"/"Frau" enthält zum dictionary hinzufügen.

Die fertige Liste mit Pandas angucken und freuen.


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
# Sachen vorbereiten

data = {'page': '1',
        'pageSize': '50',   #'1593',  <- Für vollständigen Datensatz
        'mainSearchText': '*'
       }

cookies = {'ASP.NET_SessionId': 'nt4pmcae3gesz4kk41h2lang'}

url = 'http://appl.fr.ch/ate/Person/FilterGridWithIds'
sub_base = 'http://appl.fr.ch/ate/Person/Detail/'


# Zugriff auf die Webseite

response = requests.post(url, data=data, cookies=cookies)
response_as_json = json.loads(response.text)

# For Loop

verzeichnis = []

for person in response_as_json['Data']:
    thisperson = {}
    thisperson['1 Name'] = person['DisplayName'] if not person['DisplayName'] is None else ''
    thisperson['2 ID'] = person['Id'] if not person['Id'] is None else ''
    thisperson['3 Email'] = person['Email'] if not person['Email'] is None else ''
    thisperson['4 Direktion'] = person['Roles'][0]['DirectionName'] if not person['Roles'] is None and len(person['Roles']) > 0 else ''
    thisperson['5 Sektion'] = person['UnitsDisplay'] if not person['UnitsDisplay'] is None else ''
    thisperson['6 Abteilung'] = person['UnitsExports'] if not person['UnitsExports'] is None else ''
    thisperson['7 Funktion'] = person['RolesDisplay'] if not person['RolesDisplay'] is None else ''
    
    # Uuund das Sub-Page Aufrufen.
    sub_url = sub_base + str(person['Id'])
    
    r = requests.get(sub_url)
    soup = BeautifulSoup(r.content, 'lxml')
    legend = soup.find_all('legend')[0]
    for span in legend('span'):
        span.decompose()
    
    thisperson['8 Gender'] = legend
    
    verzeichnis.append(thisperson)
    
pd.DataFrame(verzeichnis)

NameError: name 'json' is not defined

In [ ]:
pd